In [1]:
using Pkg; Pkg.activate("./binder/")

Activating environment at `~/work/cfgrib-julia-notebooks/binder/Project.toml`


# Intro

This notebook explores a number of Julia packaes which provide similar functionality to `xarrays` in python.

The main packages to be explored are:

 - [x] [AxisArrays](https://github.com/JuliaArrays/AxisArrays.jl)
 - [x] [NamedArrays](https://github.com/davidavdav/NamedArrays.jl)
 - [ ] [Images](https://github.com/JuliaImages/Images.jl) (combines [AxisArrays](https://github.com/JuliaArrays/AxisArrays.jl) and [SimpleTraits](https://github.com/mauro3/SimpleTraits.jl))

## PyCall - Loading cfgrib example data

Define some basic wrapper functions to read in data via python calls to cfgrib and convert it to Julia dictionaries

These dictionaries can then be used to attempt to re-create the python xarrays

In [2]:
using DataStructures
using Dates

In [3]:
using PyCall

cfgrib = pyimport("cfgrib")

PyObject <module 'cfgrib' from '/home/roscar/environments/cfgrib-python/lib/python3.7/site-packages/cfgrib/__init__.py'>

### Conversions

In [4]:
import Base.convert

function convert(::Type{DateTime}, np_datetime::PyObject)
    """Converts python numpy `datetime64[ns]` to Julia `DateTime`
    """
    data = np_datetime.tolist()

    epoch_start = Dates.datetime2epochms(DateTime("1970-01-01T00:00:00"))
    data = [Dates.epochms2datetime(d*1e-6 + epoch_start) for d in data]

    return data
end

function convert(::Type{Nanosecond}, np_datetime::PyObject)
    """Converts python numpy `timedelta64[ns]` to Julia `Dates.Nanosecond`
    """
    data = np_datetime.tolist()
    
    data = [Dates.Nanosecond(d) for d in data]
    
    return data
end

function convert(::Type{Dates.AbstractDateTime}, np_datetime::PyObject)
    """Dispatches PyObject Date conversion to either `DateTime` or `Nanosecond`
    based on the `dtype` value of the `PyObject`
    """
    @assert np_datetime.dtype.str in ("<M8[ns]", "<m8[ns]")
    
    if np_datetime.dtype.str == "<M8[ns]" #  M8 is for times from epoch
        return convert(DateTime, np_datetime)
    elseif np_datetime.dtype.str == "<m8[ns]" #  m8 is for time delta
        return convert(Nanosecond, np_datetime)
    end
end

convert (generic function with 232 methods)

In [5]:
function python_cfgrib_coords(cfgrib_ds::PyObject, data_variable::String)
    coordinates = OrderedDict()
    for (name, value) in cfgrib_ds.coords.items()
        coord_values = value.values
        if typeof(coord_values) == PyObject
            #  If automatic conversion failed it's (probably) a numpy
            #  datetime64[ns] or timedelta[ns] value
            coord_values = convert(Dates.AbstractDateTime, coord_values)
        end
        coordinates[name] = coord_values
    end

    return coordinates
end

function python_cfgrib_coords(path::String, data_variable::String)
    cfgrib_ds = cfgrib.open_dataset(path)
    return python_cfgrib_coords(cfgrib_ds, data_variable)
end

cfgrib_coords = python_cfgrib_coords("./data/era5-levels-members.grib", "t")

OrderedDict{Any,Any} with 7 entries:
  "number"        => [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
  "time"          => DateTime[2017-01-01T00:00:00, 2017-01-01T12:00:00, 2017-01…
  "step"          => 0 nanoseconds
  "isobaricInhPa" => [850, 500]
  "latitude"      => [90.0, 87.0, 84.0, 81.0, 78.0, 75.0, 72.0, 69.0, 66.0, 63.…
  "longitude"     => [0.0, 3.0, 6.0, 9.0, 12.0, 15.0, 18.0, 21.0, 24.0, 27.0  ……
  "valid_time"    => DateTime[2017-01-01T00:00:00, 2017-01-01T12:00:00, 2017-01…

In [6]:
function python_cfgrib_data(cfgrib_ds::PyObject, data_variable::String)
    python_shape = cfgrib_ds.get(data_variable).shape
    values = cfgrib_ds.get(data_variable).values
    
    @assert python_shape == size(values)
    
    return values
end

function python_cfgrib_data(path::String, data_variable::String)
    cfgrib_ds = cfgrib.open_dataset(path)

    return python_cfgrib_data(cfgrib_ds, data_variable)
end

cfgrib_data = python_cfgrib_data("./data/era5-levels-members.grib", "t")

println(size(cfgrib_data))

(10, 4, 2, 61, 120)


In [7]:
python_cfgrib_metadata(cfgrib_ds::PyObject, data_variable::String) = cfgrib_ds.get(data_variable).attrs

function python_cfgrib_metadata(path::String, data_variable::String)
    cfgrib_ds = cfgrib.open_dataset(path)
    
    return OrderedDict(python_cfgrib_metadata(cfgrib_ds, data_variable))
end

cfgrib_meta = python_cfgrib_metadata("./data/era5-levels-members.grib", "t")

OrderedDict{Any,Any} with 30 entries:
  "GRIB_units"                             => "K"
  "long_name"                              => "Temperature"
  "GRIB_dataType"                          => "an"
  "GRIB_totalNumber"                       => 10
  "GRIB_jScansPositively"                  => 0
  "GRIB_name"                              => "Temperature"
  "GRIB_gridType"                          => "regular_ll"
  "GRIB_Ny"                                => 61
  "GRIB_longitudeOfLastGridPointInDegrees" => 357.0
  "GRIB_stepUnits"                         => 1
  "GRIB_jPointsAreConsecutive"             => 0
  "standard_name"                          => "air_temperature"
  "GRIB_jDirectionIncrementInDegrees"      => 3.0
  "GRIB_gridDefinitionDescription"         => "Latitude/Longitude Grid"
  "GRIB_latitudeOfLastGridPointInDegrees"  => -90.0
  "GRIB_shortName"                         => "t"
  "GRIB_missingValue"                      => 9999
  "GRIB_stepType"                          => "in

In [8]:
function python_cfgrib_load(path::String, data_variable::String)
    cfgrib_ds = cfgrib.open_dataset(path)
    
    coords = python_cfgrib_coords(cfgrib_ds, data_variable)
    data   = python_cfgrib_data(cfgrib_ds, data_variable)
    meta   = python_cfgrib_metadata(cfgrib_ds, data_variable)
    
    return (data=data, coords=coords, meta=meta)
end

cfgrib_xarray_data = python_cfgrib_load("./data/era5-levels-members.grib", "t");

## AxisArrays

> This package for the Julia language provides an array type (the AxisArray) that knows about its dimension names and axis values. This allows for indexing with the axis name without incurring any runtime overhead. AxisArrays can also be indexed by the values of their axes, allowing column names or interval selections. This permits one to implement algorithms that are oblivious to the storage order of the underlying arrays. In contrast to similar approaches in Images.jl and NamedArrays.jl, **this allows for type-stable selection of dimensions and compile-time axis lookup**. It is also better suited for regularly sampled axes, like samples over time.

**NOTE**: There are ongoing extensive discussion about implementing large changes to AxisArrays in the future due to some limitations caused by the current architecture of the package. These changes should end up being mostly internal, meaning that packages depending on AxisArrays should not have to change much (if at all) to be compatible with future versions. The discussions are spread across these issues on GitHub: [AxisArrays Roadmap](https://github.com/JuliaArrays/AxisArrays.jl/issues/7), [AxisArrays Issue: Use value indexing by default](https://github.com/JuliaArrays/AxisArrays.jl/issues/84), [AxisArraysFuture Plan](https://github.com/JuliaCollections/AxisArraysFuture/issues/1),

In [9]:
using AxisArrays

In [10]:
AxisArray(1:2:10) #  AxisArray with unnamed axis

1-dimensional AxisArray{Int64,1,...} with axes:
    :row, Base.OneTo(5)
And data, a 5-element StepRange{Int64,Int64}:
 1
 3
 5
 7
 9

In [11]:
AxisArray(1:2:10, Axis{:time}(0:4)) #  AxisArray with a single named axis

1-dimensional AxisArray{Int64,1,...} with axes:
    :time, 0:4
And data, a 5-element StepRange{Int64,Int64}:
 1
 3
 5
 7
 9

In [12]:
cfgrib_xarray_data.coords

OrderedDict{Any,Any} with 7 entries:
  "number"        => [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
  "time"          => DateTime[2017-01-01T00:00:00, 2017-01-01T12:00:00, 2017-01…
  "step"          => 0 nanoseconds
  "isobaricInhPa" => [850, 500]
  "latitude"      => [90.0, 87.0, 84.0, 81.0, 78.0, 75.0, 72.0, 69.0, 66.0, 63.…
  "longitude"     => [0.0, 3.0, 6.0, 9.0, 12.0, 15.0, 18.0, 21.0, 24.0, 27.0  ……
  "valid_time"    => DateTime[2017-01-01T00:00:00, 2017-01-01T12:00:00, 2017-01…

In [13]:
size(cfgrib_xarray_data.data)

(10, 4, 2, 61, 120)

In [14]:
axisarray_example = AxisArray(
    cfgrib_xarray_data.data,
    Axis{:number}(cfgrib_xarray_data.coords["number"]),
    Axis{:time}(cfgrib_xarray_data.coords["time"]),
    Axis{:isobaricInhPa}(cfgrib_xarray_data.coords["isobaricInhPa"]),
    Axis{:latitude}(cfgrib_xarray_data.coords["latitude"]),
    Axis{:longitude}(cfgrib_xarray_data.coords["longitude"]),
)

typeof(axisarray_example)

AxisArray{Float32,5,Array{Float32,5},Tuple{Axis{:number,Array{Int64,1}},Axis{:time,Array{DateTime,1}},Axis{:isobaricInhPa,Array{Int64,1}},Axis{:latitude,Array{Float64,1}},Axis{:longitude,Array{Float64,1}}}}

In [15]:
axisarray_example

5-dimensional AxisArray{Float32,5,...} with axes:
    :number, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
    :time, DateTime[2017-01-01T00:00:00, 2017-01-01T12:00:00, 2017-01-02T00:00:00, 2017-01-02T12:00:00]
    :isobaricInhPa, [850, 500]
    :latitude, [90.0, 87.0, 84.0, 81.0, 78.0, 75.0, 72.0, 69.0, 66.0, 63.0  …  -63.0, -66.0, -69.0, -72.0, -75.0, -78.0, -81.0, -84.0, -87.0, -90.0]
    :longitude, [0.0, 3.0, 6.0, 9.0, 12.0, 15.0, 18.0, 21.0, 24.0, 27.0  …  330.0, 333.0, 336.0, 339.0, 342.0, 345.0, 348.0, 351.0, 354.0, 357.0]
And data, a 10×4×2×61×120 Array{Float32,5}:
[:, :, 1, 1, 1] =
 252.663  251.854  251.142  252.044
 252.277  251.73   250.983  252.548
 252.449  251.733  250.829  252.358
 252.283  252.258  250.811  252.494
 252.049  251.622  250.824  251.921
 252.376  252.039  251.123  252.284
 252.131  251.842  251.281  252.17 
 252.173  251.64   251.116  252.252
 251.714  251.768  251.422  252.368
 251.881  251.83   250.935  252.599

[:, :, 2, 1, 1] =
 233.31   233.139  231.737  230.701

AxisArrays has no built-in support for attributes but this can be solved with a simple wrapper type which holds the AxisArray and a dictionary:

In [16]:
struct MetaAxisArray
    array::AxisArray
    metadata::OrderedDict
end

In [17]:
#  Basic show to get a feel for what was done
function Base.show(io::IO, m::MetaAxisArray)
    axes_names = axisnames(m.array)
    axes_values = axisvalues(m.array)

    print(io, "Dimensions: ")
    [print(io, "$(a), ") for a in zip(axes_names, size.(axes_values))]

    print(io, "\nCoordinates:\n")
    [print(io, "\t$(a)\n") for a in zip(axes_names, typeof.(axes_values))]
    
    print(io, "Data: $(size(m.array))\n")
    
    print(io, "Metadata:\n")
    [print(io, "\t$(md)\n") for md in m.metadata]
end

In [18]:
meta_axisarray_example = MetaAxisArray(axisarray_example, cfgrib_xarray_data.meta)

Dimensions: (:number, (10,)), (:time, (4,)), (:isobaricInhPa, (2,)), (:latitude, (61,)), (:longitude, (120,)), 
Coordinates:
	(:number, Array{Int64,1})
	(:time, Array{DateTime,1})
	(:isobaricInhPa, Array{Int64,1})
	(:latitude, Array{Float64,1})
	(:longitude, Array{Float64,1})
Data: (10, 4, 2, 61, 120)
Metadata:
	"GRIB_units" => "K"
	"long_name" => "Temperature"
	"GRIB_dataType" => "an"
	"GRIB_totalNumber" => 10
	"GRIB_jScansPositively" => 0
	"GRIB_name" => "Temperature"
	"GRIB_gridType" => "regular_ll"
	"GRIB_Ny" => 61
	"GRIB_longitudeOfLastGridPointInDegrees" => 357.0
	"GRIB_stepUnits" => 1
	"GRIB_jPointsAreConsecutive" => 0
	"standard_name" => "air_temperature"
	"GRIB_jDirectionIncrementInDegrees" => 3.0
	"GRIB_gridDefinitionDescription" => "Latitude/Longitude Grid"
	"GRIB_latitudeOfLastGridPointInDegrees" => -90.0
	"GRIB_shortName" => "t"
	"GRIB_missingValue" => 9999
	"GRIB_stepType" => "instant"
	"GRIB_numberOfPoints" => 7320
	"GRIB_NV" => 0
	"GRIB_latitudeOfFirstGridPointInDegrees

## NamedArrays

> We would want to have the possibility to give each row/column/... in an Array names, as well as the array dimensions themselves. This could be used for pretty-printing, indexing, and perhaps even some sort of dimension-checking in certain matrix computations.
> 
> In all other respects, a NamedArray should behave the same as the underlying AbstractArray.
> 
> A NamedArray should adhere to the interface definition of an AbstractArray itself, if there are cases where this is not true, these should be considered bugs in the implementation of NamedArrays.

In [19]:
using NamedArrays

In [20]:
n = NamedArray([1 2 3; 4 5 6], (["one", "two"], [:a, :b, :c]))

2×3 Named Array{Int64,2}
A ╲ B │ :a  :b  :c
──────┼───────────
one   │  1   2   3
two   │  4   5   6

In [21]:
n[:, :a]

2-element Named Array{Int64,1}
A   │ 
────┼──
one │ 1
two │ 4

In [22]:
NamedArray([1 3; 2 4], ( ["a", "b"], ["c", "d"] ), ("Rows", "Cols"))

2×2 Named Array{Int64,2}
Rows ╲ Cols │ c  d
────────────┼─────
a           │ 1  3
b           │ 2  4

In [23]:
only_data_coords = copy(cfgrib_xarray_data.coords)
delete!(only_data_coords, "step")
delete!(only_data_coords, "valid_time")

OrderedDict{Any,Any} with 5 entries:
  "number"        => [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
  "time"          => DateTime[2017-01-01T00:00:00, 2017-01-01T12:00:00, 2017-01…
  "isobaricInhPa" => [850, 500]
  "latitude"      => [90.0, 87.0, 84.0, 81.0, 78.0, 75.0, 72.0, 69.0, 66.0, 63.…
  "longitude"     => [0.0, 3.0, 6.0, 9.0, 12.0, 15.0, 18.0, 21.0, 24.0, 27.0  ……

In [24]:
collect(o for o in only_data_coords)

5-element Array{Pair{String,B} where B,1}:
        "number" => [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]                                                                                                       
          "time" => DateTime[2017-01-01T00:00:00, 2017-01-01T12:00:00, 2017-01-02T00:00:00, 2017-01-02T12:00:00]                                         
 "isobaricInhPa" => [850, 500]                                                                                                                           
      "latitude" => [90.0, 87.0, 84.0, 81.0, 78.0, 75.0, 72.0, 69.0, 66.0, 63.0  …  -63.0, -66.0, -69.0, -72.0, -75.0, -78.0, -81.0, -84.0, -87.0, -90.0]
     "longitude" => [0.0, 3.0, 6.0, 9.0, 12.0, 15.0, 18.0, 21.0, 24.0, 27.0  …  330.0, 333.0, 336.0, 339.0, 342.0, 345.0, 348.0, 351.0, 354.0, 357.0]    

In [25]:
namedarray_example = NamedArray(
    cfgrib_xarray_data.data[:, :, 1, 1, 1],
    (only_data_coords["number"], only_data_coords["time"]),
    ("number", "time")
)

10×4 Named Array{Float32,2}
number ╲ time │ 2017-01-01T00:00:00  …  2017-01-02T12:00:00
──────────────┼────────────────────────────────────────────
0             │             252.663  …              252.044
1             │             252.277                 252.548
2             │             252.449                 252.358
3             │             252.283                 252.494
4             │             252.049                 251.921
5             │             252.376                 252.284
6             │             252.131                  252.17
7             │             252.173                 252.252
8             │             251.714                 252.368
9             │             251.881  …              252.599

In [26]:
namedarray_example = NamedArray(
    cfgrib_xarray_data.data,
    Tuple(values(only_data_coords)),
    Tuple(names(only_data_coords))
)

10×4×2×61×120 Named Array{Float32,5}

[:, :, isobaricInhPa=850, latitude=90.0, longitude=0.0] =
number ╲ time │ 2017-01-01T00:00:00  …  2017-01-02T12:00:00
──────────────┼────────────────────────────────────────────
0             │             252.663  …              252.044
⋮                                 ⋮  ⋱                    ⋮
9             │             251.881  …              252.599

[:, :, isobaricInhPa=500, latitude=90.0, longitude=0.0] =
number ╲ time │ 2017-01-01T00:00:00  …  2017-01-02T12:00:00
──────────────┼────────────────────────────────────────────
0             │              233.31  …              230.701
⋮                                 ⋮  ⋱                    ⋮
9             │             233.147  …              230.401

[:, :, isobaricInhPa=850, latitude=87.0, longitude=0.0] =
number ╲ time │ 2017-01-01T00:00:00  …  2017-01-02T12:00:00
──────────────┼────────────────────────────────────────────
0             │              252.61  …              253.349
⋮     

In [27]:
names(namedarray_example)

5-element Array{Array{T,1} where T,1}:
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]                                                                                                       
 DateTime[2017-01-01T00:00:00, 2017-01-01T12:00:00, 2017-01-02T00:00:00, 2017-01-02T12:00:00]                                         
 [850, 500]                                                                                                                           
 [90.0, 87.0, 84.0, 81.0, 78.0, 75.0, 72.0, 69.0, 66.0, 63.0  …  -63.0, -66.0, -69.0, -72.0, -75.0, -78.0, -81.0, -84.0, -87.0, -90.0]
 [0.0, 3.0, 6.0, 9.0, 12.0, 15.0, 18.0, 21.0, 24.0, 27.0  …  330.0, 333.0, 336.0, 339.0, 342.0, 345.0, 348.0, 351.0, 354.0, 357.0]    

In [28]:
dimnames(namedarray_example)

5-element Array{String,1}:
 "number"       
 "time"         
 "isobaricInhPa"
 "latitude"     
 "longitude"    

As before, no attributes but they can be added in

## Images

Two main packages from the Images.jl ecosystem may be interesting:

[ImageMetadata.jl](https://juliaimages.org/latest/imagemetadata/):

> ImageMetadata (a package incorporated into Images) allows you to add metadata to images: for example, the date and time at which it was collected, identifiers for the location or subject, etc. This metadata is stored as a dictionary, and the ImageMeta type combines properties of arrays and Dict.

[ImaegAxes.jl](https://juliaimages.org/ImageAxes.jl/stable/):

> While images can often be represented as plain Arrays, sometimes additional information about the "meaning" of each axis of the array is needed. For example, in a 3-dimensional MRI scan, the voxels may not have the same spacing along the z-axis that they do along the x- and y-axes, and this fact should be accounted for during the display and/or analysis of such images. Likewise, a movie has two spatial axes and one temporal axis; this fact may be relevant for how one performs image processing.
> 
> This package combines features from AxisArrays and SimpleTraits to provide a convenient representation and programming paradigm for dealing with such images.